In [1]:
import pandas as pd
import seaborn as sns
from datetime import date

In [2]:
def find_sheet_name(sheet_names):
    iip_sheet_names = list(filter(lambda x: "spcn" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(filter(lambda x: "sp cn" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(
        filter(lambda x: "spcn" in x.lower() and "thang" in x.lower(), sheet_names)
    )

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]
    
    iip_sheet_names = list(filter(lambda x: "sp" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    # iip_sheet_names = list(filter(lambda x: "thang" in x.lower(), iip_sheet_names))

    # if len(iip_sheet_names) == 1:
    #     return iip_sheet_names[0]

    if len(sheet_names) == 43:
        return "9"

    # iip_sheet_names = list(filter(lambda x: "gtcn" in x.lower(), sheet_names))

    # if len(iip_sheet_names) == 1:
    #     return iip_sheet_names[0]

    raise RuntimeError(sheet_names, iip_sheet_names)

In [3]:
en_to_vn = {
    "Coal (pure)": "Than đá (than sạch)",
    "Extracted crude oil": "Dầu mỏ thô khai thác",
    "Natural gas (in the form of air)": "Khí đốt thiên nhiên dạng khí",
    "Liquidized gas (LPG)": "Khí hoá lỏng (LPG)",
    "Petroleum": "Xăng, dầu",
    "Aluminium": "Alumin",
    "Processed fishery products": "Thuỷ hải sản chế biến",
    "Fresh milk": "Sữa tươi",
    "Powder milk": "Sữa bột",
    "Refined sugar": "Đường kính",
    "Monosodium glutamate": "Bột ngọt",
    "Animal feed": "Thức ăn cho gia súc",
    "Aquatic feed": "Thức ăn cho thủy sản",
    "Beer": "Bia",
    "Cigarettes": "Thuốc lá điếu",
    "Textile fabric from natural yarn": "Vải dệt từ sợi tự nhiên",
    "Textile fabric from polyester or artificial yarn": "Vải dệt từ sợi tổng hợp hoặc sợi nhân tạo",
    "Clothes": "Quần áo mặc thường",
    "Leather footgear": "Giày, dép da",
    "Urea fertilizer": "Phân U rê",
    "N.P.K mixed fertilizer": "Phân hỗn hợp N.P.K",
    "Chemical paint": "Sơn hoá học ",
    "Cement": "Xi măng",
    "Crude steel, iron": "Sắt, thép thô",
    "Laminated steel": "Thép cán",
    "Steel bars and corners": "Thép thanh, thép góc",
    "Mobile phone": "Điện thoại di động",
    "Phone accessories": "Linh kiện điện thoại",
    "Television": "Tivi ",
    "Automobile": "Ô tô",
    "Motorbike": "Xe máy",
    "Generated electricity": "Điện sản xuất",
    "Running water": "Nước máy thương phẩm",
}

In [4]:
def get_spcn(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df.reset_index(inplace=True, drop=True)

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "Than đá (than sạch)")
                    | ((df[df.columns[0]] == "Coal (pure)"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "Coal (pure)"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    df = df.drop(columns=[df.columns[1]])

    if len(df.columns) == 5 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[3]])
        df.columns = ["Ngành", "YOY"]
        df["YTD-YOY"] = df["YOY"]

    if len(df.columns) == 6:
        df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[3]])
        df.columns = ["Ngành", "YOY", "YTD-YOY"]

    if len(df.columns) == 8:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[3],
                df.columns[6],
                df.columns[7],
            ]
        )
        df.columns = ["Ngành", "YOY", "YTD-YOY"]

    assert len(df.columns) == 3, df.columns

    df["YOY"] = df["YOY"] - 100
    df["YTD-YOY"] = df["YTD-YOY"] - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("datas/2017-03-29-Bieu-3-2017-1.xlsx")

get_spcn(xl, find_sheet_name(xl.sheet_names), "2017-03-29")

,Ngành,YOY,YTD-YOY,Date
4,Than đá (than sạch),1.578896,-5.552874,2017-03-29
5,Dầu mỏ thô khai thác,-14.117647,-14.851485,2017-03-29
6,Khí đốt thiên nhiên dạng khí,1.926409,-8.913114,2017-03-29
7,Khí hoá lỏng (LPG),2.579533,-3.938969,2017-03-29
8,Thuỷ hải sản chế biến,15.0,10.166908,2017-03-29
9,Sữa tươi,2.054285,12.734145,2017-03-29
10,Sữa bột,15.486985,5.525086,2017-03-29
11,Đường kính,-6.2,-7.367391,2017-03-29
12,Bột ngọt,3.133669,5.192788,2017-03-29
13,Thức ăn cho gia súc,16.0,9.346951,2017-03-29


In [5]:
import os
import contextlib

dfs_spcn_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2013:
            continue

        xl = pd.ExcelFile(file_path)

        df_spcn = get_spcn(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_spcn_origin.append(df_spcn)
    except Exception as e:
        print(file_name)
        raise e

In [6]:
dfs_spcn = pd.concat(dfs_spcn_origin, ignore_index=True, axis=0)
dfs_spcn["Date"] = pd.to_datetime(dfs_spcn["Date"], dayfirst=False)
dfs_spcn

,Ngành,YOY,YTD-YOY,Date
0,Than đá (than sạch),15.497406,15.497406,2024-01-29
1,Dầu mỏ thô khai thác,-2.209945,-2.209945,2024-01-29
2,Khí đốt thiên nhiên dạng khí,0.055076,0.055076,2024-01-29
3,Khí hoá lỏng (LPG),16.769912,16.769912,2024-01-29
4,"Xăng, dầu",25.754859,25.754859,2024-01-29
...,...,...,...,...
4302,Xe chở khách,-8.391633,-8.391633,2013-01-29
4303,Xe tải,38.241708,38.241708,2013-01-29
4304,Xe máy,28.609839,28.609839,2013-01-29
4305,Điện sản xuất,25.748727,25.748727,2013-01-29


In [7]:
convert_nganh = {
    "Than đá (than sạch)": "Than đá (than sạch)",
    "Dầu mỏ thô khai thác": "Dầu mỏ thô khai thác",
    "Khí đốt thiên nhiên dạng khí": "Khí đốt thiên nhiên dạng khí",
    "Khí hoá lỏng (LPG)": "Khí hoá lỏng (LPG)",
    "Xăng, dầu": "Xăng, dầu",
    "Alumin": "Alumin",
    "Thuỷ hải sản chế biến": "Thuỷ hải sản chế biến",
    "Sữa tươi": "Sữa tươi",
    "Sữa bột": "Sữa bột",
    "Đường kính": "Đường kính",
    "Bột ngọt": "Bột ngọt",
    "Thức ăn cho gia súc": "Thức ăn cho gia súc",
    "Thức ăn cho thủy sản": "Thức ăn cho thủy sản",
    "Bia": "Bia",
    "Thuốc lá điếu": "Thuốc lá điếu",
    "Vải dệt từ sợi tự nhiên": "Vải dệt từ sợi tự nhiên",
    "Vải dệt từ sợi tổng hợp hoặc sợi nhân tạo": "Vải dệt từ sợi tổng hợp hoặc sợi nhân tạo",
    "Quần áo mặc thường": "Quần áo mặc thường",
    "Giày, dép da": "Giày, dép da",
    "Phân U rê": "Phân U rê",
    "Phân hỗn hợp N.P.K": "Phân hỗn hợp N.P.K",
    "Sơn hoá học": "Sơn hoá học",
    "Xi măng": "Xi măng",
    "Sắt, thép thô": "Sắt, thép thô",
    "Thép cán": "Thép cán",
    "Thép thanh, thép góc": "Thép thanh, thép góc",
    "Điện thoại di động": "Điện thoại di động",
    "Linh kiện điện thoại": "Linh kiện điện thoại",
    "Tivi": "Tivi",
    "Ô tô": "Ô tô",
    "Xe máy": "Xe máy",
    "Điện sản xuất": "Điện sản xuất",
    "Nước máy thương phẩm": "Nước máy thương phẩm",
    "Ti vi": "Tivi",
    "Xăng, dầu các loại": "Xăng, dầu",
    "Bia các loại": "Bia",
    "Ti vi các loại": "Tivi",
    "Xăng dầu": "Xăng, dầu",
    "Tivi các loại": "Tivi",
    "Sơn hoá học ": "Sơn hoá học",
    "Tivi ": "Tivi",
    "Dầu gội đầu, dầu xả": "Dầu gội đầu, dầu xả",
    "Sữa tắm, sữa rửa mặt": "Sữa tắm, sữa rửa mặt",
    "Phân Ure": "Phân U rê",
    "Dầu mỏ thô khai thác trong nước": "Dầu mỏ thô khai thác",
    "Phân hỗn hợp (N, P, K)": "Phân hỗn hợp N.P.K",
    "Sơn hoá học các loại": "Sơn hoá học",
    "Bột giặt và các chế phẩm dùng để tẩy, rửa": "Bột giặt và các chế phẩm dùng để tẩy, rửa",
    "Gạch xây bằng đất nung": "Gạch xây bằng đất nung",
    "Xe chở khách": "Xe chở khách",
    "Xe tải": "Xe tải",
    "Giày, dép, ủng bằng da giả cho người lớn": "Giày, dép da",
}

dfs_spcn["Ngành"] = dfs_spcn["Ngành"].map(convert_nganh)
dfs_spcn

,Ngành,YOY,YTD-YOY,Date
0,Than đá (than sạch),15.497406,15.497406,2024-01-29
1,Dầu mỏ thô khai thác,-2.209945,-2.209945,2024-01-29
2,Khí đốt thiên nhiên dạng khí,0.055076,0.055076,2024-01-29
3,Khí hoá lỏng (LPG),16.769912,16.769912,2024-01-29
4,"Xăng, dầu",25.754859,25.754859,2024-01-29
...,...,...,...,...
4302,Xe chở khách,-8.391633,-8.391633,2013-01-29
4303,Xe tải,38.241708,38.241708,2013-01-29
4304,Xe máy,28.609839,28.609839,2013-01-29
4305,Điện sản xuất,25.748727,25.748727,2013-01-29


In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_spcn[
    (dfs_spcn["Ngành"] == "Thuỷ hải sản chế biến")
    | (dfs_spcn["Ngành"] == "Sữa tươi")
    | (dfs_spcn["Ngành"] == "Sữa bột")
    # | (dfs_spcn["Ngành"] == "Đường kính")
    # | (dfs_spcn["Ngành"] == "Quần áo mặc thường")
    # | (dfs_spcn["Ngành"] == "Phân U rê")
    # | (dfs_spcn["Ngành"] == "Phân hỗn hợp N.P.K")
    # | (dfs_spcn["Ngành"] == "Sắt, thép thô")
    # | (dfs_spcn["Ngành"] == "Thép cán")
    # | (dfs_spcn["Ngành"] == "Thép thanh, thép góc")
    # | (dfs_spcn["Ngành"] == "Điện thoại di động")
    # | (dfs_spcn["Ngành"] == "Linh kiện điện thoại")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)
# fig.update_layout(yaxis_range=[-100,600])

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

